In [1]:
import python_utils as utils
import sys
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.optimize import minimize, curve_fit
from scipy import stats
from statsmodels.stats.multitest import multipletests
from scipy.signal import correlate
from scipy.interpolate import interp1d

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 13


# SIGMOIDPULSE0_002 t_mid = 13000 duration = 10 s

In [2]:
parseg = 250
overlap = 0.5

neglect_t = 3000
t_mid = 13000
t_mid2 = 23000
sigm_par = 0.002

t_tot = 33000

dd_low = 0.850

function = "sigmoidpulse"

In [3]:
sigm_par_poiss = "{:.5f}".format(sigm_par)

Dd = "0.950"
subnets = "STN"
simulations = [str(i) for i in range(32)]


In [4]:
Dd_values = []
current_value = dd_low
while current_value <= float(Dd):
    Dd_values.append(round(current_value, 4))
    current_value += 0.0025

print(Dd_values)

[0.85, 0.8525, 0.855, 0.8575, 0.86, 0.8625, 0.865, 0.8675, 0.87, 0.8725, 0.875, 0.8775, 0.88, 0.8825, 0.885, 0.8875, 0.89, 0.8925, 0.895, 0.8975, 0.9, 0.9025, 0.905, 0.9075, 0.91, 0.9125, 0.915, 0.9175, 0.92, 0.9225, 0.925, 0.9275, 0.93, 0.9325, 0.935, 0.9375, 0.94, 0.9425, 0.945, 0.9475, 0.95]


In [5]:
simulations_folder_path = f"./build/output/n1/{function}_{t_mid}.00_{sigm_par_poiss}_0.00_{Dd}_1.00_1.00_1.6/"
print(simulations_folder_path)

./build/output/n1/sigmoidpulse_13000.00_0.00200_0.00_0.950_1.00_1.00_1.6/


# EXpected PSD

In [6]:
####################### SIMULATION EXPECTED DATA #######################################
cleaned_path = simulations_folder_path.replace("./build/output/n1/", "").replace("_1.00_1.00_1.6/", "")
filename = f"power_{cleaned_path}_{parseg}.txt"

power = np.loadtxt(simulations_folder_path + filename, skiprows=1)
time = np.arange(0, len(power[0]), 1)

In [7]:
df_attesi = []

for i in range(32):
    # Crea un DataFrame con i dati
    df_i = pd.DataFrame({'Tempo': time, 'Valore': power[i]})

    # Calcola la media mobile e assegna i risultati al tempo centrale
    window_size = parseg
    df_i['Media_Mobile'] = df_i['Valore'].rolling(window=window_size, center=True).mean()

    # Rimuovi le righe con valori mancanti (prime e ultime finestre)
    df_i = df_i.dropna()

    # Rimuovi i primi neglect_t elementi
    df_i = df_i.iloc[neglect_t:]

    # Aggiungi il DataFrame alla lista
    df_attesi.append(df_i)

# Stampa il DataFrame risultante
#print(df_attesi)

In [8]:
media = []
error = []
tt = []

for i in range(0, len(df_attesi[0]['Tempo']), int(parseg - parseg*overlap)): # qui metti 0. overlap
    media.append(np.mean([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi]))
    error.append(np.std([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi]))
    tt.append(i + int(parseg*overlap))


In [9]:
tt = np.array(tt)/1000

# Dynamic simulation PSD

In [10]:
####################### DATI SIMULAZIONE #######################################
cleaned_path = simulations_folder_path.replace("./build/output/n1/", "").replace("_1.00_1.00_1.6/", "")
filename = f"all_pow_t{cleaned_path}_{parseg}.txt"

all_pow_t = np.loadtxt(simulations_folder_path + filename, skiprows=1)


In [11]:
pow_t_mean = np.mean(all_pow_t, axis=0)
pow_error = np.std(all_pow_t, axis=0)

num_elements = len(pow_t_mean)
t = np.linspace(parseg*overlap/1000, num_elements * parseg*overlap/1000, num_elements)


# Delay error

In [12]:
%matplotlib widget
# Definizione della funzione sigmoide
def sigmoidpulse_fit(x, sigm_par, t_1, t_2, up, down):
    return up + 1 / (1 + np.exp(-sigm_par * (x - t_1))) * (down - up) - 1 / (1 + np.exp(-sigm_par * (x - t_2))) * (down - up) 

# Valori iniziali per i parametri del fit
initial_guess = [sigm_par, t_mid -neglect_t, t_mid2-neglect_t, 0.26, 0]

In [13]:
ttt = np.arange(0,t_tot - neglect_t,1)/1000

In [14]:
params2, covariance2 = curve_fit(sigmoidpulse_fit, tt*1000, media, p0=initial_guess, sigma=error)
y_fit2 = sigmoidpulse_fit(ttt*1000, params2[0], params2[1], params2[2], params2[3], params2[4]) 

In [15]:
cross_corr = []
all_delay = []

In [16]:
for i in range(32):

    # Creazione dell'oggetto di interpolazione
    interpolator1 = interp1d(t[:], all_pow_t[i], kind='linear', fill_value="extrapolate")
    interpolator2 = interp1d(t[:], media, kind='linear', fill_value="extrapolate")

    # Applicazione dell'interpolatore ai 30000 punti
    pow_t_mean_interpolated1 = interpolator1(ttt)
    pow_t_mean_interpolated2 = interpolator2(ttt)



    #mi concentro sulle parti in pendenza metti 3 per lente e 8 per veloci
    width = 3.1 / sigm_par /1000
    # Converti t_1 e t_2 in secondi per corrispondere a `ttt`
    t_1_sec = params2[1] / 1000
    t_2_sec = params2[2] / 1000

    # Calcola gli intervalli
    interval_1 = [t_1_sec - width, t_1_sec + width]
    interval_2 = [t_2_sec - width, t_2_sec + width]


    # Funzione per estrarre un segmento data una serie temporale e un intervallo
    def extract_segment(time_series, t_values, interval):
        mask = (t_values >= interval[0]) & (t_values <= interval[1])
        return time_series[mask]

    segment_1_pow = extract_segment(pow_t_mean_interpolated1, ttt, interval_1)
    segment_2_pow = extract_segment(pow_t_mean_interpolated1, ttt, interval_2)

    segment_1_fit = extract_segment(pow_t_mean_interpolated2, ttt, interval_1)
    segment_2_fit = extract_segment(pow_t_mean_interpolated2, ttt, interval_2)


    #tiling procedure
    ripetizioni = 100
    concatenated_pow = np.concatenate([segment_1_pow, segment_2_pow] * ripetizioni)
    concatenated_fit = np.concatenate([segment_1_fit, segment_2_fit] * ripetizioni)
    
    if i == 4:
        %matplotlib widget
        plt.figure(figsize=(10, 6))
        plt.plot( concatenated_pow, label='Simulated PSD')
        plt.plot( concatenated_fit, label='Expected PSD')
        #plt.title(f' for Interval around {params2[1]:.0f}')
        plt.ylabel('Mean β PSD [u.a.]')
        plt.xlabel('Time [ms]')
        plt.xlim(0,width*4000)
        plt.legend()
        plt.grid(True)
        plt.show()


    cross_corr_3 = correlate(concatenated_pow - np.mean(concatenated_pow), concatenated_fit - np.mean(concatenated_fit), mode='full', method='auto')
    lags_3 = np.arange(-len(concatenated_pow) + 1, len(concatenated_fit))

    # Filtra l'array dei lag per considerare solo quelli tra 0 e 500
    indices_within_range = (lags_3 >= -2000) & (lags_3 <= 2000)
    filtered_lags = lags_3[indices_within_range]
    filtered_cross_corr = cross_corr_3[indices_within_range]

    # Trova l'indice del valore massimo di cross-correlazione nel sottoinsieme filtrato
    max_corr_index_filtered = np.argmax(filtered_cross_corr)

    # Usa questo indice per trovare il corrispondente valore di lag
    delay = filtered_lags[max_corr_index_filtered]
    
    cross_corr.append(filtered_cross_corr)
    all_delay.append(delay)

    # Stampa il lag corrispondente al massimo nella cross-correlazione, limitato all'intervallo specificato
    print("Lag corrispondente al massimo:", delay)

Lag corrispondente al massimo: 81
Lag corrispondente al massimo: 113
Lag corrispondente al massimo: 333
Lag corrispondente al massimo: -1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Lag corrispondente al massimo: 320
Lag corrispondente al massimo: 268
Lag corrispondente al massimo: -13
Lag corrispondente al massimo: 283
Lag corrispondente al massimo: 68
Lag corrispondente al massimo: 326
Lag corrispondente al massimo: 234
Lag corrispondente al massimo: 279
Lag corrispondente al massimo: 206
Lag corrispondente al massimo: 772
Lag corrispondente al massimo: -412
Lag corrispondente al massimo: 205
Lag corrispondente al massimo: 264
Lag corrispondente al massimo: -51
Lag corrispondente al massimo: 17
Lag corrispondente al massimo: 118
Lag corrispondente al massimo: 755
Lag corrispondente al massimo: 409
Lag corrispondente al massimo: -36
Lag corrispondente al massimo: -12
Lag corrispondente al massimo: 54
Lag corrispondente al massimo: -63
Lag corrispondente al massimo: 177
Lag corrispondente al massimo: 144
Lag corrispondente al massimo: 480
Lag corrispondente al massimo: 767
Lag corrispondente al massimo: 463
Lag corrispondente al massimo: 653


In [17]:
delay = round(np.mean(all_delay))
mean_cross_corr = np.mean(cross_corr, axis=0)/ripetizioni
std_cross_corr = np.std(cross_corr, axis=0)/ripetizioni

# Crea la figura e gli assi una volta
plt.figure(figsize=(9, 6))

# Traccia l'andamento medio
plt.plot(filtered_lags, mean_cross_corr, label='Cross-Correlation')
plt.scatter(delay, mean_cross_corr[filtered_lags == delay], color='red', zorder=5) 
# Aggiungi l'errore sfumato come banda di confidenza
plt.fill_between(filtered_lags, mean_cross_corr - std_cross_corr, mean_cross_corr + std_cross_corr, color='gray', alpha=0.5, label='Standard deviation of the mean')

# Imposta le etichette e i limiti dell'asse
plt.xlabel('Lag [ms]')
plt.ylabel('Cross-Correlation')
plt.xlim(-2000, 2000)
plt.ylim(0)
plt.grid(True)

# Aggiungi una legenda per chiarire i componenti del grafico
plt.legend(loc='upper left')

# Mostra il grafico
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
std_value_approx = np.round(std_cross_corr[filtered_lags == delay])[0]

# Stampiamo il valore approssimato
print("delay:", delay ,"±", std_value_approx)

delay: 225 ± 16.0


In [19]:
%matplotlib widget
# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize=(12, 5))

# Utilizza la funzione errorbar per tracciare i dati con errori associati
ax1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)

# Plot dei dati originali
ax1.plot(t, pow_t_mean, label='Intensità simulazione', color='orange')
#ax1.set_yscale('log')

# Aggiungi il terzo set di dati traslati
plt.errorbar(tt + delay/1000, media, yerr=error, fmt='o', label='Media Mobile con errore')#errore non shiftato??????


# Etichette e legenda per l'asse x e y
ax1.set_xlabel('Time [s] [s]')
ax1.set_ylabel('Intensità media')
ax1.legend()  # Assicurati di avere tutte le etichette delle leggende necessarie

plt.title(f'Intensità nel Range β per τ = {1/sigm_par/1000} [s] nparseg = {parseg}')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Traslo pulse atteso e ricalcolo intensità attesa

In [20]:
power_shift = []
for i in range(32):
    p = np.roll(power[i], round(delay))
    power_shift.append(p)

In [21]:
all_pow_t_transposed = list(zip(*all_pow_t))

In [22]:
df_attesi = []

for i in range(32):
    # Crea un DataFrame con i dati
    df_i = pd.DataFrame({'Tempo': time[i], 'Valore': power_shift[i]})

    # Calcola la media mobile e assegna i risultati al tempo centrale
    window_size = parseg
    df_i['Media_Mobile'] = df_i['Valore'].rolling(window=window_size, center=True).mean()

    # Rimuovi le righe con valori mancanti (prime e ultime finestre)
    df_i = df_i.dropna()

    # Rimuovi i primi neglect_t elementi
    df_i = df_i.iloc[neglect_t:]

    # Aggiungi il DataFrame alla lista
    df_attesi.append(df_i)

# Stampa il DataFrame risultante
#print(df_attesi)

In [23]:
media_s = []
error_s = []
tt = []

for i in range(0, len(df_attesi[0]['Tempo']), int(parseg - parseg*overlap)): # qui metti 0. overlap
    media_s.append(np.mean([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi]))
    error_s.append(np.std([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi]))
    tt.append(i + int(parseg*overlap))


tt = np.array(tt)/1000

In [24]:
pow_attesa_shift = []
for i in range(0, len(df_attesi[0]['Tempo']), int(parseg - parseg*overlap)): # qui metti 0. overlap
    pow_attesa_shift.append([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi])

In [25]:
# Esegui il test t indipendente (two-sample t-test)
t_statistic = []
p_value = []
for i in range(len(pow_attesa_shift)):
    ts, v = stats.ttest_ind(all_pow_t_transposed[i], pow_attesa_shift[i])
    t_statistic.append(ts)
    p_value.append(v)


In [26]:
p_=multipletests(p_value, alpha=0.05, method='fdr_bh', maxiter=1, is_sorted=False, returnsorted=False)

In [27]:
# %matplotlib widget

# Crea un array di valori temporali
time_values = np.arange(parseg*overlap/1000, len(p_value) * parseg*overlap/1000 + parseg*overlap/1000, parseg*overlap/1000)

# Creare le figure per i p-values
fig, (axPvalue1, axPvalue2) = plt.subplots(nrows=2, ncols=1, figsize=(12, 10))

# Plot per la prima figura
axPvalue1.plot(time_values, pow_t_mean, marker='o', color='orange', markerfacecolor='black', label='Dynamic simulation β PSD')
axPvalue1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)
axPvalue1.errorbar(tt + delay/1000, media, yerr=error, fmt='o', label='expected Mean β PSD')

true_indices = np.where(p_[0])[0]
if true_indices.size > 0:  # Controlla se true_indices contiene almeno un elemento
    axPvalue1.axvline(x=time_values[true_indices[0]], color='green', alpha=0.2, label='p-value < 0.05')
    for index in true_indices:
        axPvalue1.axvline(x=time_values[index], color='green', alpha=0.2)

axPvalue1.text(0.01, 0.95, '(a)', transform=axPvalue1.transAxes, fontsize=16, fontweight='bold', va='top')

#axPvalue1.set_yscale('log')
axPvalue1.set_ylabel('Mean β PSD [u.a.]')
axPvalue1.set_title(f'Simulation and expected values with delay vs P-value  τ = {1/sigm_par/1000} [s] $y_L$ = {10*(dd_low - 0.85):.2f}')# nparseg = {parseg}')
axPvalue1.legend()

# Plot per la seconda figura
axPvalue2.plot(time_values, p_[1], marker='.', linestyle='-', markerfacecolor='black', label='P-values')
axPvalue2.set_yscale('log')
axPvalue2.axhline(y=0.05, color='red', linestyle='-', label='0.05 threshold')

if true_indices.size > 0:  # Ripete il controllo per la seconda figura
    axPvalue2.axvline(x=time_values[true_indices[0]], color='green', alpha=0.2, label='p-value < 0.05')
    for index in true_indices:
        axPvalue2.axvline(x=time_values[index], color='green', alpha=0.2)

axPvalue2.text(0.01, 0.95, '(b)', transform=axPvalue2.transAxes, fontsize=16, fontweight='bold', va='top')

axPvalue2.set_xlabel('Time [s]')
axPvalue2.set_ylabel('P-value')
axPvalue2.legend()

# Mostra le figure
plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …